In [49]:
from scipy.io import loadmat
from scipy.io import savemat
import pandas as pd
import numpy as np
import transformations  as tr
import random as rand
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13893123679371085174
]


In [3]:
def generatePairs(frames):
    framePairs = [] #vector to hold image pairs

    for x in range(len(frames)): #x represents the first frame in the pair
        for y in range(len(frames)-3-x): #y represents second frame in the pair
            pair = [(x+1),(x+y+4),np.stack((frames[x], frames[x+y+3]), axis=-1)] #generate pair
            framePairs.append(pair)
    return framePairs

In [4]:
def generateGoodData(goodData):
    extraGood = []
    for x in range(len(goodData)):
        extra = translatePair(goodData[x][1],goodData[x][2],goodData[x][0],goodData[x][3],1,1)
        extraGood = extraGood + extra
    return extraGood

In [5]:
def translatePair(frame1,frame2,patient,data,steps,label):

    frame1 = frame1-1 #get first frame position from pair object
    frame2 = frame2-1 #get second frame position from pair object
    JAdd = data.shape[0] #Get size of original frame to keep translation size consistent
    IAdd = data.shape[1] #Get size of original frame to keep translation size consistent

    baseCut = 0.1 #use same base cut as is used in original frame generation
    step = baseCut/steps #set step size to be as large as possible for given number of steps
    pairs = [] #vector to hold newly generated pairs

    #adjust how the frame is cropped from the original data to simulate translation
    for x in range(-steps,steps+1): 
        for y in range(-steps,steps+1):
            lowerI = int((baseCut+x*step)*256)
            upperI = lowerI+IAdd
            lowerJ = int((baseCut+y*step)*2064)
            upperJ = lowerJ+JAdd

            first = rawData[patient][1][lowerJ:upperJ,lowerI:upperI,frame1]
            second = rawData[patient][1][lowerJ:upperJ,lowerI:upperI,frame2]

            pairs.append([patient,frame1,frame2,np.stack((first,second), axis=-1),label]) #append translated pair to pairs vector
    return pairs

In [6]:
def normalize(min,max,array):
    test[test < min] = min
    test[test > max] = max
    array = tr.unit_vector(array)
    return array

In [64]:
#Directory containing the RF scan data
dataFolder = 'C:/Users/Lucas/OneDrive - The University of Western Ontario/Documents/GitHub/SE4450-project-code/Preprocessing/'

#Vector to hold all training data sets
rawData = []

#Loading in the matlab file information
rawMatFile = loadmat('P39-W2-S4')
#Contains just the RF data pulled from the .mat file
P39W2S4 = rawMatFile['rf1']
rawData.append([0,P39W2S4])

#repeat above steps for other RF files
rawMatFile = loadmat('P39-W4-S6')
P39W4S6 = rawMatFile['rf1']
rawData.append([1,P39W4S6])
rawMatFile = loadmat('P82-W0-S2')
P82W0S2 = rawMatFile['rf1']
# rawData.append(['P82W0S2',P82W0S2])
rawMatFile = loadmat('P87-W0-S3')
P87W0S3 = rawMatFile['rf1']
rawData.append([2,P87W0S3])
rawMatFile = loadmat('P87-W2-S4')
P87W2S4 = rawMatFile['rf1']
rawData.append([3,P87W2S4])
rawMatFile = loadmat('P90-W0-S4')
P90W0S4 = rawMatFile['rf1']
rawData.append([4,P90W0S4])
rawMatFile = loadmat('P94-W1-S3')
P94W1S3 = rawMatFile['rf1']
rawData.append([5,P94W1S3])

In [8]:
#leave a slice of each edge of the scan from being black
cutTop = 0.1
cutLeft = 0.1
cutBottom = 0.1
cutRight = 0.1

lowerI = int(cutLeft*256)
upperI = int((1-cutRight)*256)
lowerJ = int(cutTop*2064)
upperJ = int((1-cutBottom)*2064)

allFrames = [] #vector to hold individual cropped frames

for y in range(len(rawData)):
    frames = []
    for x in range(rawData[y][1].shape[2]): #range is how many frames there are in the set
        print(rawData[y][1].shape)
        frames.append(rawData[y][1][lowerJ:upperJ,lowerI:upperI,x]) #Take inner subset of frame to allow for translation
    allFrames.append([rawData[y][0],frames])

pairs = []
for x in range(len(allFrames)):
    pairs.append([allFrames[x][0],generatePairs(allFrames[x][1])]) #add generated pairs to list
print(pairs[5][1][0][2].shape)

#PAIRS VARIABLE GUIDE
#First value = Holds which patient set is being referred to
#Second value = Holds either patient set name [0] or generated pairs for set
#Third value = Holds all pairs for patient set
#Fourth value = holds specific pairs for patient set
#pairs[0][1][0][2].shape <- references the shape of the third pair of frames for a certain patients data

(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 23)
(2064, 256, 19)
(2064, 256, 19)
(2064, 256, 19)
(2064, 256, 19)
(2064, 256, 19)
(2064, 256, 19)
(2064, 256, 19)
(2064, 256, 19)
(2064, 256, 19)
(2064, 256, 19)
(2064, 256, 19)
(2064, 256, 19)
(2064, 256, 19)
(2064, 256, 19)
(2064, 256, 19)
(2064, 256, 19)
(2064, 2

In [9]:
#Create Vector of stacked pairs with frame number and patient ID for easy labeling
data = []
for x in range(len(pairs)):
    for y in range(len(pairs[x][1])):
        data.append([pairs[x][0],pairs[x][1][y][0],pairs[x][1][y][1],pairs[x][1][y][2]])

print(len(data))
for x in range(2):
    print(data[x][0],data[x][1],data[x][2],data[x][3].shape)

998
0 1 4 (1651, 205, 2)
0 1 5 (1651, 205, 2)


In [90]:
#Read in label data
DFP39W2S4 = pd.read_excel('Labels.xlsx', sheet_name='P39-W2-S4', engine='openpyxl')
DFP39W4S6 = pd.read_excel('Labels.xlsx', sheet_name='P39-W4-S6', engine='openpyxl')
DFP87W0S3 = pd.read_excel('Labels.xlsx', sheet_name='P87-W0-S3', engine='openpyxl')
DFP87W2S4 = pd.read_excel('Labels.xlsx', sheet_name='P87-W2-S4', engine='openpyxl')
DFP90W0S4 = pd.read_excel('Labels.xlsx', sheet_name='P90-W0-S4', engine='openpyxl')
DFP94W1S3 = pd.read_excel('Labels.xlsx', sheet_name='P94-W1-S3', engine='openpyxl')

In [11]:
# + DFP82W0S2['Label axial'].tolist() wrong shape?
#create labels for training data
labels = DFP39W2S4['Label axial'].tolist() + DFP39W4S6['Label axial'].tolist()  + DFP87W0S3['Label axial'].tolist() + DFP87W2S4['Label axial'].tolist() + DFP90W0S4['Label axial'].tolist() + DFP94W1S3['Label axial'].tolist()
print(len(labels))

998


In [12]:
labeledData = []
for x in range(len(labels)):
    labeledData.append([data[x][0],data[x][1],data[x][2],data[x][3],labels[x]])

In [13]:
goodData = []
badData = []
for x in range(len(labeledData)):
    #Setting meh labels to 0
    if labeledData[x][4] == 0.5:
        labeledData[x][4] = 0

    if labeledData[x][4] > 0:
        goodData.append(labeledData[x])
    else:
        badData.append(labeledData[x])
print(len(goodData))
print(len(badData))
# for x in range(len(goodData)):
#     print(goodData[x][3].shape)
goodData = generateGoodData(goodData)
goodData = rand.sample(goodData,len(badData))
print(len(goodData))
print(len(badData))
balancedData = goodData+badData
print(len(balancedData))

159
839
839
839
1678


In [14]:
def train_preprocessing(data, label):
    data = tf.expand_dims(data, axis=3)
    return data, label


def validation_preprocessing(data, label):
    data = tf.expand_dims(data, axis=3)
    return data, label

In [15]:
# Split data in the ratio 80-20 for training and validation.
x_train, x_test, y_train, y_test = train_test_split([x[3] for x in balancedData], [x[4] for x in balancedData], test_size=0.20, random_state=42)

# print(len(y_train))

In [16]:
for x in range(len(x_train)):
    x_train[x] = tf.convert_to_tensor(x_train[x], np.float32)

for x in range(len(x_test)):
    x_test[x] = tf.convert_to_tensor(x_test[x], np.float32)

In [17]:
# Define data loaders.
train_loader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
validation_loader = tf.data.Dataset.from_tensor_slices((x_test, y_test))

batch_size = 2
# Augment the on the fly during training.
train_dataset = (
    train_loader.shuffle(len(x_train))
    .map(train_preprocessing)
    .batch(batch_size)
    .prefetch(2)
)
# Only rescale.
validation_dataset = (
    validation_loader.shuffle(len(x_test))
    .map(validation_preprocessing)
    .batch(batch_size)
    .prefetch(2)
)

In [18]:
def get_model(width=1651, height=205, depth=2):
    """Build a 3D convolutional neural network model."""

    inputs = keras.Input((width, height, depth, 1))

    x = layers.Conv3D(filters=64, kernel_size=(3,3,2), activation="relu")(inputs)
    x = layers.MaxPool3D(pool_size=(2,2,1))(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=64, kernel_size=(3,3,1), activation="relu")(x)
    x = layers.MaxPool3D(pool_size=(2,2,1))(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=128, kernel_size=(3,3,1), activation="relu")(x)
    x = layers.MaxPool3D(pool_size=(2,2,1))(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=256, kernel_size=(3,3,1), activation="relu")(x)
    x = layers.MaxPool3D(pool_size=(2,2,1))(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=512, kernel_size=(3,3,1), activation="relu")(x)
    x = layers.MaxPool3D(pool_size=(2,2,1))(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(units=512, activation="relu")(x)
    x = layers.Dropout(0.3)(x)

    outputs = layers.Dense(units=1, activation="sigmoid")(x)

    # Define the model.
    model = keras.Model(inputs, outputs, name="3dcnn")
    return model


# Build model.
model = get_model(width=1651, height=205, depth=2)
model.summary()


Model: "3dcnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1651, 205, 2, 1)] 0         
_________________________________________________________________
conv3d (Conv3D)              (None, 1649, 203, 1, 64)  1216      
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 824, 101, 1, 64)   0         
_________________________________________________________________
batch_normalization (BatchNo (None, 824, 101, 1, 64)   256       
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 822, 99, 1, 64)    36928     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 411, 49, 1, 64)    0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 411, 49, 1, 64)    256   

In [33]:
# Compile model.
initial_learning_rate = 0.0001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)
model.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
    metrics=["acc"],
)

# Define callbacks.
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    "3d_image_classification.h5", save_best_only=True
)
early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_acc", patience=15)

# Train the model, doing validation at the end of each epoch
epochs = 50
model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=epochs,
    shuffle=True,
    verbose=2,
    callbacks=[checkpoint_cb, early_stopping_cb],
)


Epoch 1/50
671/671 - 9562s - loss: 0.6684 - acc: 0.6051 - val_loss: 0.4926 - val_acc: 0.8006
Epoch 2/50
671/671 - 7098s - loss: 0.5805 - acc: 0.6863 - val_loss: 0.5201 - val_acc: 0.6488
Epoch 3/50
671/671 - 6586s - loss: 0.5177 - acc: 0.7377 - val_loss: 0.5698 - val_acc: 0.7024
Epoch 4/50
671/671 - 6461s - loss: 0.4800 - acc: 0.7601 - val_loss: 0.3955 - val_acc: 0.8304
Epoch 5/50
671/671 - 6473s - loss: 0.4485 - acc: 0.7779 - val_loss: 0.3712 - val_acc: 0.8542
Epoch 6/50
671/671 - 6460s - loss: 0.4323 - acc: 0.7869 - val_loss: 0.4088 - val_acc: 0.7649
Epoch 7/50
671/671 - 6469s - loss: 0.4149 - acc: 0.8085 - val_loss: 0.5401 - val_acc: 0.7560
Epoch 8/50
671/671 - 6543s - loss: 0.3842 - acc: 0.8189 - val_loss: 0.4084 - val_acc: 0.7976
Epoch 9/50
671/671 - 6785s - loss: 0.3758 - acc: 0.8361 - val_loss: 0.4404 - val_acc: 0.7500
Epoch 10/50
671/671 - 12788s - loss: 0.3493 - acc: 0.8539 - val_loss: 0.4967 - val_acc: 0.7768
Epoch 11/50


In [37]:
model.load_weights("3d_image_classification.h5")
prediction = model.predict(np.expand_dims(np.expand_dims(x_test[32], axis=3),axis=0))[0]
scores = [1 - prediction[0], prediction[0]]

class_names = ["A bad pair", "An optimal pair"]
for score, name in zip(scores, class_names):
    print(
        "This model is %.2f percent confident that frame pair is %s"
        % ((100 * score), name)
    )

This model is 1.55 percent confident that frame pair is A bad pair
This model is 98.45 percent confident that frame pair is An optimal pair


In [61]:
print(y_test[32])
predictions = []
print(len(y_test))
for x in range(len(y_test)):
    if (x%10==0):
        print(x)
    if (predictLabel(x)>0.9):
        predictions.append(1)
    else:
        predictions.append(0)

1
336
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330


In [54]:
def predictLabel(number):
    prediction = model.predict(np.expand_dims(np.expand_dims(x_test[number], axis=3),axis=0))[0]
    return prediction

In [62]:
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
print(tn, fp, fn, tp)
print((tn+tp)/336)

164 0 77 95
0.7708333333333334


In [52]:
confusion_matrix(y_test, predictions)

array([[149,  15],
       [ 31, 141]], dtype=int64)

In [91]:
def preprocessData(data):
    height = data.shape[0]
    width =  data.shape[1]
    lowerH = int((height-1651)/2)
    upperH = lowerH+1651
    lowerW = int((width-205)/2)
    upperW = lowerW + 205
    print(data.shape)

    frames = []
    for x in range(data.shape[2]): #range is how many frames there are in the set
        frames.append(data[lowerJ:upperJ,lowerI:upperI,x]) #cutdata to match size expected by model

    pairs = generatePairs(frames)
    return pairs

In [100]:
processed = preprocessData(P82W0S2)
print(len(processed))

(2592, 256, 19)
136


In [139]:
predicitons = []
print(len(processed))
for x in range(len(processed)):
    if (x%25==0):
        print(x)
    if ((model.predict(np.expand_dims(np.expand_dims(processed[x][2], axis=3),axis=0))[0])>0.5):
        predicitons.append(1)
    else:
        predicitons.append(0)

136
0
25
50
75
100
125
136


In [137]:
DFP82W0S2 = pd.read_excel('Labels.xlsx', sheet_name='P82-W0-S2', engine='openpyxl')
DFP82W0S2 = DFP82W0S2['Label axial'].tolist()
for x in range(len(DFP82W0S2)):
    #Setting 0.5 labels to 0
    if DFP82W0S2[x] == 0.5:
        DFP82W0S2[x] = 0
print(type(DFP82W0S2))
print(len(DFP82W0S2))

<class 'list'>
136


In [140]:
tn, fp, fn, tp = confusion_matrix(DFP82W0S2, predicitons).ravel()
print(tn, fp, fn, tp)

118 0 18 0
